In [1]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts
# from manual_encoding import action_encoding, click_view_encoding, meta_encoding
# from hotel2vec import hotel2vec
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
    

In [ ]:
# train = load_data('train', nrows=5000000)

In [ ]:
# train.action_type.unique()

In [ ]:
# train.action_type.nunique()

In [15]:
mode = 'train'
nrows = 1000000
# df = load_data('train', nrows=nrows)
df = prepare_data(mode, convert_action_type=False, nrows=nrows, recompute=True)
df = compute_session_fts(df, mode)

[05-14 07:56:44 - utils-106 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data
[05-14 07:56:44 - create_model_inputs-17 - flogger - INFO] raw train shape: (999,995, 12)
[05-14 07:56:44 - clean_session-60 - preprocess_sessions - INFO] Dropping duplicates
[05-14 07:56:44 - clean_session-18 - remove_duplicates - INFO] Before dropping duplicates df shape: (999,995, 12)
[05-14 07:56:45 - clean_session-21 - remove_duplicates - INFO] After dropping duplicates df shape: (421,117, 12)
[05-14 07:56:45 - clean_session-62 - preprocess_sessions - INFO] Cliping session dataframe up to last click out (if there is clickout)
[05-14 07:57:08 - clean_session-65 - preprocess_sessions - INFO] Filtering out sessions without clickouts, reference, or clickout is nan
[05-14 07:57:08 - clean_session-66 - preprocess_sessions - INFO] train length before filtering no click-out, or nan reference and click-out sessions: 362,672
[05-14 07:57:56 - clean_sess

In [16]:
sids = df.session_id.unique()

In [29]:
rid = np.random.choice(sids, 1)[0]
rid

'fbdea6023f5b0'

In [30]:
df[df.session_id==rid]

,user_id,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices,timestamp_session_duration,timestamp_dwell_time_prior_clickout,current_filters_last_filters,session_id_size
251268,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:03:03,1,clickout item,NaN,1649739,1649739|478616|1330207|152438|478256|104507|10...,69|77|106|52|68|36|45|70|76|63|69|57|31|39|67|...,615.0,38.0,Best Value,6
251269,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:11:11,2,change of sort order,NaN,interaction sort button,NaN,NaN,615.0,38.0,Best Value,6
251270,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:11:13,3,change of sort order,NaN,price and recommended,NaN,NaN,615.0,38.0,Best Value,6
251271,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:11:13,4,filter selection,Best Value,Best Value,NaN,NaN,615.0,38.0,Best Value,6
251272,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:12:40,5,search for item,NaN,104832,NaN,NaN,615.0,38.0,Best Value,6
251273,QVTSQD8X27Y5,fbdea6023f5b0,2018-11-05 01:13:18,6,clickout item,NaN,1189712,478616|1330207|478256|2755492|1241263|1474801|...,503|805|397|238|147|361|175|209|196|245|198|23...,615.0,38.0,Best Value,6


In [20]:
df[df.session_id==rid].impressions[df[df.session_id==rid].impressions.notna()].values

array(['1188848|2296240|1828119|1826199|3976206|2674500|2817444|5418974|9150584|6409612|753741|1655703|5964878|1814881|1045654|2087440|2569860|1710127|2047409|508396|1776237|1395549|1852481|4011134|1773599',
       '1188848|2296240|1828119|1826199|3976206|2674500|2817444|5418974|9150584|6409612|753741|1655703|5964878|1814881|1045654|2087440|2569860|1710127|2047409|508396|1776237|1395549|1852481|4011134|1773599',
       '1188848|2296240|1828119|1826199|3976206|2674500|2817444|5418974|9150584|6409612|753741|1655703|5964878|1814881|1045654|2087440|2569860|1710127|2047409|508396|1776237|1395549|1852481|4011134|1773599',
       '1188848|2296240|1828119|1826199|3976206|2674500|2817444|5418974|9150584|6409612|753741|1655703|5964878|1814881|1045654|2087440|2569860|1710127|2047409|508396|1776237|1395549|1852481|4011134|1773599'],
      dtype=object)

In [6]:
from create_model_inputs import create_action_type_mapping

In [7]:
m, _ = create_action_type_mapping()

[05-14 07:53:35 - create_model_inputs-25 - create_action_type_mapping - INFO] Load action_types mapping from existing: ./cache/action_types_mapping.npy


In [8]:
m

{'search for poi': 0,
 'interaction item image': 1,
 'clickout item': 2,
 'interaction item info': 3,
 'interaction item deals': 4,
 'search for destination': 5,
 'filter selection': 6,
 'interaction item rating': 7,
 'search for item': 8,
 'change of sort order': 9}

### session_id_size

In [ ]:
# session_size
df.session_id_size.hist(bins=200)

In [ ]:
np.log(df.session_id_size).hist(bins=200)

In [ ]:
logged = np.log(df.session_id_size)

In [ ]:
((logged - logged.mean())/logged.std()).hist(bins=200)

In [ ]:
((df.session_id_size - df.session_id_size.mean())/df.session_id_size.std()).hist(bins=200)

In [ ]:
df.session_id_size.mean()

In [ ]:
df.session_id_size.std()

### timestamp_dwell_time_prior_clickout

In [ ]:
df.timestamp_dwell_time_prior_clickout.hist(bins=100)

In [ ]:
df.timestamp_dwell_time_prior_clickout.describe()

In [ ]:
df.timestamp_dwell_time_prior_clickout.clip(upper=60**2).describe()

In [ ]:
df.timestamp_dwell_time_prior_clickout.clip(upper=60**2).hist(bins=100)

In [ ]:
np.log1p(df.timestamp_dwell_time_prior_clickout.clip(upper=60**2)).hist(bins=50)

In [ ]:
qt = 0.999
df.timestamp_dwell_time_prior_clickout.quantile(qt)

In [ ]:
df.timestamp_dwell_time_prior_clickout.quantile(qt)/60

### last_ref_ind

In [ ]:
from utils import get_logger
logger = get_logger('create_model_inputs')

In [ ]:
# logger.info('Lower case current filters and split to list')
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# logger.info('Split prices str to list and convert to int')
# df['prices'] = df['prices'].str.split('|')
df = df.groupby('session_id').last().reset_index()

# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])
logger.info('Pad 0s for prices length shorter than 25')
# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))

logger.info('Split impression str to list of impressions')
df['impressions'] = df['impressions'].str.split('|')
# logger.info('Convert impression str to int')
# df['impressions'] = df['impressions'].apply(lambda x: [int(i) for i in x])

if mode == 'train':
    logger.info('Assign target')
    logger.info('Convert reference id to int')
    df['reference'] = df['reference'].astype(int)

    # filter out nan rows with reference_id not in impressions list, since if the true target in test
    # is not in the impression list then it would not get evaluated
    def assign_target(row):
        ref = row['reference']
        imp = list(row['impressions'])
        if ref in imp:
            return imp.index(ref)
        else:
            return np.nan

    df['target'] = df.apply(assign_target, axis=1)
    logger.info('Remove the ones whose reference id is not in impression list')
    # drop the ones whose reference is not in the impression list
    df = df[df['target'].notna()].reset_index(drop=True)
    df['target'] = df['target'].astype(int)
    logger.info(f"Target distribution: \n{pd.value_counts(df['target']).head()}")

logger.info('Assign location of previous reference id')

# def assign_last_ref_id(row):
#     ref = row['reference_last_reference_id']
#     # although reference_id got converted to int, but the reference_last_reference_id was calculated
#     # when it was still str value, so here we look up the index in str of impressions
#     imp = [str(i) for i in row['impressions']]
#     if pd.isna(ref):
#         return np.nan
#     else:
#         if ref in imp:
#             return imp.index(ref) + 1
#             # return (imp.index(ref) + 1) / len(imp)
#         else:
#             return np.nan

In [ ]:
# %%time
# df['last_ref_ind'] = df.apply(assign_last_ref_id, axis=1)

In [ ]:
# df.last_ref_ind.hist(bins=100)

In [ ]:
# np.log(df.last_ref_ind).hist(bins=100)

In [ ]:
# ((df.last_ref_ind - df.last_ref_ind.mean())/df.last_ref_ind.std()).hist(bins=100)

In [ ]:
# (df.last_ref_ind/25).hist(bins=100)

### prices

In [ ]:
prices = np.concatenate(df.prices.values)

In [ ]:
pdf = pd.DataFrame({'p': prices})

In [ ]:
pdf.p.hist(bins=100)

In [ ]:
np.log1p(pdf.p).hist(bins=100)

In [ ]:
((pdf.p - pdf.p.mean())/pdf.p.std()).hist(bins=100)

In [ ]:
pdf.p.quantile(0.9)

In [ ]:
pdf.p.quantile(0.99999)

In [ ]:
pdf.p.describe()

### look at data
---

In [ ]:
df.head()

In [ ]:
# %%time
# train_raw = load_data('train')
# train_raw['timestamp'] = train_raw['timestamp'].apply(lambda t: datetime.datetime.utcfromtimestamp(t))
# train_raw['nfilters'] = train_raw['current_filters'].str.split('|').str.len()
# train = train_raw[(train_raw.action_type == 'clickout item') & (train_raw.impressions.notna())].copy()

In [ ]:
# fprint(train, 'train')

In [ ]:
# (train_raw.action_type=='clickout item').sum()

In [ ]:
# %%time
# # also get test
# test = load_data('test')#, nrows=1000000)
# fprint(test, 'raw test')
# test = test[(test.action_type == 'clickout item') & (test.impressions.notna()) & (test.reference.notna())].reset_index(drop=True)
# fprint(test, 'test after')

In [ ]:
# test_sub = load_data('submission_popular')

In [ ]:
%%time
def session_duration(x):
    return x.max() - x.min()
# session_fts = train.groupby('session_id').agg({'session_id': 'size', 'timestamp': session_duration})
session_fts = train.groupby('session_id').size().reset_index(name='session_size')

In [ ]:
%%time
train_last = train.groupby('session_id').last().reset_index()

In [ ]:
fprint(train_last, 'trian_last')

In [ ]:
train_last = pd.merge(train_last, session_fts, on='session_id')

### user_id

In [ ]:
# train_last.groupby('user_id').size()
train_last.user_id.value_counts().hist(bins=100)

In [ ]:
# train_last.groupby('user_id').size()
np.log(train_last.user_id.value_counts()).hist(bins=100)

In [ ]:
train_last.user_id.nunique()

In [ ]:
train_user_ids = train_last.user_id.unique()

In [ ]:
test_user_ids = test.user_id.unique()

In [ ]:
sub_ids = test_sub.user_id.unique()

In [ ]:
len(set(train_user_ids).intersection(sub_ids))

In [ ]:
sub_ids.shape

In [ ]:
26913/sub_ids.shape[0]

### what are the ones click on index 0 look like and the ones with rest index

In [ ]:
one_sids = df[df.target==0].session_id
rest_sids = df[df.target!=0].session_id

In [ ]:
oid = np.random.choice(one_sids, 1)[0]
rid = np.random.choice(rest_sids, 1)[0]
oid, rid

In [ ]:
print('target:', df[df.session_id==oid].target.values[0], '| session_size:',
     df[df.session_id==oid].session_size.values[0], '| user freq:',
     df[df.session_id==oid].user_id.size, '\n')
for i in train[train.session_id==oid].reset_index(drop=True).values:
    print(i)

In [ ]:
ps = np.concatenate(train[train.session_id==oid].prices.drop_duplicates().str.split('|').values)
ps = [int(i) for i in ps]
ps = np.sort(ps)
ps

---

In [ ]:
print('target:', train_last[train_last.session_id==rid].target.values[0], '| session_size:',
     train_last[train_last.session_id==rid].session_size.values[0], '| user freq:',
     train_last[train_last.session_id==rid].user_id.size, '\n')
for i in train[train.session_id==rid].reset_index(drop=True).values:
    print(i)

In [ ]:
ps = train[train.session_id==rid].prices.drop_duplicates().str.split('|').iloc[-1]
ps = np.sort([int(i) for i in ps])
ps

### check raw

In [ ]:
train_raw[train_raw.session_id==oid].reset_index(drop=True)

In [ ]:
train_raw[train_raw.session_id==rid].reset_index(drop=True)

In [ ]:
1541231265 - 1541231272

In [ ]:
len(set(train_raw.session_id.unique()).intersection(test.session_id.unique()))

In [ ]:
train = prepare_data('train', nrows=1000000)

### current_filters

In [ ]:
train_last.shape

In [ ]:
filters_counts = train_last.current_filters.value_counts()

In [ ]:
filters_counts.head()

In [ ]:
train_last.current_filters.nunique()

In [ ]:
train_last['cfs'] = train_last.current_filters.str.split('|')

In [ ]:
all_cfs = train_last[train_last.cfs.notna()].cfs.values
all_cfs = [j for i in all_cfs for j in i]

In [ ]:
all_cfs_counts = pd.value_counts(all_cfs)
all_cfs_counts.head()

In [ ]:
pd.value_counts(all_cfs, normalize=True).cumsum().head(20)

In [ ]:
all_cfs_counts.index[pd.DataFrame({'cfs': all_cfs_counts.index}).cfs.str.contains(r'[^(e)]sort', case=False)]

In [ ]:
all_cfs_counts.hist(bins=20)

In [ ]:
all_cfs_counts.shape

In [ ]:
# all_cfs_counts.index.values

In [ ]:
train_last.current_filters.isna().sum()

In [ ]:
train_last.shape

In [ ]:
655653/train_last.shape[0]

In [ ]:
train_last['nimps'] = train_last.impressions.str.split('|').str.len()

What is the length of the impressions for these with nan current_filters?

In [ ]:
train_last.loc[train_last.current_filters.isna(), 'nimps'].value_counts(normalize=True).head()

In [ ]:
train_last.nimps.value_counts(normalize=True).head()

In [ ]:
train_last.loc[train_last.current_filters.notna(), 'nimps'].value_counts(normalize=True).head()

What is the size of the session for these with nan current_filters?

In [ ]:
train_last.loc[train_last.current_filters.isna(), 'session_size'].value_counts(normalize=True).head()

In [ ]:
train_last.loc[train_last.current_filters.notna(), 'session_size'].value_counts(normalize=True).head()

In [ ]:
train_last.session_size.value_counts(normalize=True).head()